In [1]:
from brew_builder import BrewBuild, search_db, menu_select
import numpy as np

import sqlite3

con = sqlite3.connect('database/default_db.sqlite')


grain_bill = np.array([[9, 0.25, 0],
                       [122, 1, 0],
                       [93, 1, 1],
                       [90, 1, 1],
                       [13, 1.5, 0],
                       [24, 0.5, 0]])

hop_bill = np.array([[62, 1, 45],
                     [21, 0.5, 20]])

yeast = 3

target_volume = 2.5

boil_volume = 1.75

mash_temp = 165

build = BrewBuild(grain_bill, hop_bill, yeast, target_volume,
                               boil_volume, mash_temp, con, boil_time=45,
                               mash_efficiency=48, style=64, mash_volume=1.5)

In [2]:
build.calc_OG()

1.056

In [3]:
build.calc_FG()

1.017

In [4]:
build.calc_ABV(build.calc_OG(), build.calc_FG())

5.12

In [5]:
build.calc_color()

42.0

In [6]:
build.calc_IBU()

27.2

In [7]:
build.build_recipe('Oatmeal_Stout_Test.csv')

In [8]:
menu_select('hop', con)

interactive(children=(Text(value='Dry', description='hop:', placeholder='Type something'), Output()), _dom_cla…

In [9]:
build.interactive_sheet()

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='text', value='Summary'), Cell(co…